# 收集人臉資料

## 功能：

#### 1. 錄製一段 30s 影片 ( 影片長度可使用 frame_count 控制 )
#### 2. 將有人臉的影像取出
#### 3. 截取臉的範圍
#### 4. 將人臉圖片轉成 32x32 

In [1]:
'''
Last modified time: 2019/06/04
'''
import cv2
import os
import time
from cv2_face_detection import *

# open USB webcam
video_width = 640
video_height = 480
#cap = cv2.VideoCapture(0)   # device number 0
#cap = cv2.VideoCapture(1)   # device number 1
#cap.set(cv2.CAP_PROP_FRAME_WIDTH, video_width)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, video_height)
#print("capture device is open: " + str(cap.isOpened()))
# print('CV_CAP_PROP_BRIGHTNESS ', cap.get(cv2.CAP_PROP_BRIGHTNESS))    # BRIGHTNESS: 128(Default)
# print('CV_CAP_PROP_FPS ', cap.get(cv2.CAP_PROP_FPS))      #FPS: 30(Default)

# 影片輸出編碼方式 MP4V
#fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

image_size = 128

In [2]:
# --- read video name function --- #
def read_img(location):  # location = data/train
    l_x_train = []
    l_y_train = []
    video_list = os.listdir(location)  # label_list = ['aa', 'bb', ...]
    for i in video_list:  # loop all directory
        sub_dir = os.path.join(location, i)  # sub_dir = data/train/aa, data/train/bb, ...
        if os.path.isfile(sub_dir):  # ignore txt or other file
            continue

        img_label = i.split('_')[-1]
        # img_label = i
        for picture in os.listdir(sub_dir):  # picture = .png ...
            img_num += 1
            # print("     ", picture)
            image = cv2.imread(os.path.join(sub_dir, picture), 0)  # open .png data
            # print("     ", np.array(image).shape)
            image = np.array(image).astype(np.float32)
            image = image / 255  # normalize in 0 ~ 1
            l_x_train.append(image)
            l_y_train.append(write_label_txt(img_label))

    return l_x_train, l_y_train, img_num

In [3]:
def video2image(v_name_list):
    '''
    Divide the video frame into images.

    input:

        v name list: a list contains videos full file path
    '''
    print("[INFO] run video2image ")
    for idx in range(0, len(v_name_list)):
        v_name = v_name_list[idx]

        img_out_dir = "./image/" + v_name
        img_out_face_dir = "./image_face/" + v_name
        img_out_128_dir = "./image_128/" + v_name

        if not os.path.isdir(img_out_dir):
            os.mkdir(img_out_dir)
        if not os.path.isdir(img_out_face_dir):
            os.mkdir(img_out_face_dir)
        if not os.path.isdir(img_out_128_dir):
            os.mkdir(img_out_128_dir)

        print()
        # video name: D20190606_231806_GN_10_Mark.mp4
        print("video name:", v_name + '.mp4')
        video_cap = cv2.VideoCapture('./video/' + v_name + '.mp4')
        success, image = video_cap.read()   # read video
        if not success:
            print("read video error.")
        else:
            print("read video success.")
        
        print("Start dividing the video frame into images...")
        frame_count = 0
        while success:
            # save frame as png file
            cv2.imwrite(img_out_dir + "/" + v_name + "_frame_%d.png" % frame_count, image)
            
            # find face
            find_face, image_face = image_format_face(image)
            if find_face:
                # save face image
                cv2.imwrite(img_out_face_dir + "/" + v_name + "_frame_%d.png" % frame_count, image_face)

                image_face_gray = cv2.cvtColor(image_face, cv2.COLOR_BGR2GRAY)   # GRAY
                image_face_gray_128 = cv2.resize(image_face_gray, (image_size, image_size), interpolation=cv2.INTER_CUBIC)  # resize
                # save face gray 128 * 128 image
                cv2.imwrite(img_out_128_dir + "/" + v_name + "_frame_%d.png" % frame_count, image_face_gray_128)

            success, image = video_cap.read()
            frame_count += 1
        print("total frame number: %d." % frame_count)


def image_format_face(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)     # GRAY
    find_face, f_loc = face_detection(gray_img)
    image_fc = []
    if find_face:
        #f_loc = box_reduce(f_loc, 30)
        image_fc = face_cut(f_loc, image)
    
    return find_face, image_fc


In [4]:
def check_user_input(u_name):
    '''
    Text interaction to check user(file) name

    input:

        u name: string, user name(first is null)
    '''

    u_name = input("Please input user name [{}]: ".format(u_name)) or u_name
    while u_name == "":     # Prevent no input
        u_name = input("Please input user name [{}]: ".format(u_name)) or u_name
    
    r_angle = ""
    while not r_angle.isdigit():    # Prevent no digit
        r_angle = input("Please input user rotation angle [{}]: ".format('10')) or '10'

    return r_angle, u_name


def check_output_dir():
    '''
    Check output dir
    '''
    dir_list = ['./video', './image', './image_face', './image_32']

    for p in dir_list:
        if not os.path.isdir(p):
            os.mkdir(p)



In [5]:
if __name__ == '__main__':
    check_output_dir()
    
    print("=============================================\n")
    print("產生資料：\n")
    print("image/      (從影片收集到的 640x480 影像)\n")
    print("image_32/   (收集人臉切割後 32x32 影像)\n")
    print("image_face/ (收集的人臉影像-原始大小)\n")
    print("video/      (錄到的 15s 影片)\n\n")
    
    print("控制方法 ！\n")
    print("Step01: 輸入姓名, 按 Enter \n")
    print("Step02: 輸入角度, 按 Enter \n")
    print("Step03: Click 顯示畫面, 按下 空白鍵, 開始錄 30s\n")
    print("系統自動將影片切割收集人臉\n\n")

    print("=============================================\n\n\n")
    video_name_list = []
    Back_ground = "GN"
    user_name = ''
    
 
    # release USB web camera
    #cap.release()

    # # if read video false, modify video_name_list here
    # video_name_list = ['D20190501_113338_GN_10_Mia',
    #                    'D20190501_113443_GN_10_Mia',
    #                    'D20190501_113509_GN_10_Mia', 
    #                    'D20190501_113549_GN_10_Mia']
    # video_name_list = ['D20190604_174455_GN_10_test',
    #                    'D20190604_174649_GN_10_test'] D20190610_000000_GN_10_Vivi
    
    #video_name_list.append(video_name)
    #video2image(video_name_list)
        # --- global variable --- #
    video_parent_dir = "./video"
    video_list = []
    for video in os.listdir(video_parent_dir):  # video = .mp4 ...
        print(video)
        video_list.append(video[:-4])
        
    video2image(video_list)
    #video_list.append(video)
    




產生資料：

image/      (從影片收集到的 640x480 影像)

image_32/   (收集人臉切割後 32x32 影像)

image_face/ (收集的人臉影像-原始大小)

video/      (錄到的 15s 影片)


控制方法 ！

Step01: 輸入姓名, 按 Enter 

Step02: 輸入角度, 按 Enter 

Step03: Click 顯示畫面, 按下 空白鍵, 開始錄 30s

系統自動將影片切割收集人臉





D20200518_160828_GN_10_Vincent.mp4
D20200518_161058_GN_10_Vincent.mp4
D20200518_162417_GN_10_Allen.mp4
D20200518_162546_GN_10_Allen.mp4
D20200518_163153_GN_10_Mark.mp4
D20200518_163309_GN_10_Mark.mp4
D20200526_102520_GN_10_Forbes.mp4
D20200526_102647_GN_10_Forbes.mp4
[INFO] run video2image 

video name: D20200518_160828_GN_10_Vincent.mp4
read video success.
Start dividing the video frame into images...
total frame number: 1800.

video name: D20200518_161058_GN_10_Vincent.mp4
read video success.
Start dividing the video frame into images...
total frame number: 1800.

video name: D20200518_162417_GN_10_Allen.mp4
read video success.
Start dividing the video frame into images...
total frame number: 1800.

video name: D20200518_162546_GN_10_Allen.mp4
rea